In [1]:
# Import dependencies
import pandas as pd
import os
import requests
import gmaps
# Import Google Maps API key
from config import gKey


In [2]:
# Import csv data as a DataFrame
# ------------------------------------

fileToLoad = os.path.join("data", "WeatherPy_database.csv")
cityDataDF = pd.read_csv(fileToLoad)
cityDataDF.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Ribeira Grande,PT,38.52,-28.70,68.7,82,8,15.82,clear sky,0.0,0.0
1,1,Punta Arenas,CL,-53.15,-70.92,33.8,86,43,8.05,light snow,0.0,0.0
2,2,Ranaghat,IN,23.18,88.58,82.4,94,75,4.70,haze,0.0,0.0
3,3,Qaqortoq,GL,60.72,-46.03,51.8,57,100,4.70,light rain,0.0,0.0
4,4,Kodiak,US,57.79,-152.41,57.2,62,90,9.17,overcast clouds,0.0,0.0


In [3]:
# Get Customer preferences
# ----------------------------

# If wrong answer is given and maxTries reached, the default value would be returned
def requestTemperature(request: str, default: int, maxTries: int) -> float:
    wrongAnswer = True
    temperature = default
    tries = 1
    warningText =""
    while wrongAnswer and tries <= maxTries:
        question = input(f"{warningText}What is the {request} temperature you would like for your trip? ")
        try:
            temperature = float(question)
            wrongAnswer = False
        except ValueError:
            warningText = "Wrong value, please retry. "
            pass
        tries +=1
    return temperature

# If wrong answer is given and maxTries reached, False would be returned by default
def requestYesNoAnswer(action: str, maxTries: int) -> bool:
    wrongAnswer = True
    answer = False
    tries = 1
    warningText = ""
    while wrongAnswer and tries <= maxTries:
        question = input(f"{warningText}Do you want it to be {action}? (yes/no) ")
        if (question.lower() == "yes") or (question.lower() == "y"):
            answer = True
            wrongAnswer = False
        elif (question.lower() == "no") or (question.lower() == "n"):
            wrongAnswer = False
        else:
            warningText = "Wrong answer, please retry. "
        tries +=1
    return answer

In [4]:

# Ask the customer to add a minimum and maximum temperature value.
minTempInput = requestTemperature(request="minimum",default=0, maxTries=3)
maxTempInput = requestTemperature(request="maximum",default=100, maxTries=3)

# Ask the customer if they want rain or snow.
rainDesired = requestYesNoAnswer(action="raining",maxTries=3)
snowDesired = requestYesNoAnswer(action="snowing",maxTries=3)


What is the minimum temperature you would like for your trip? 78
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [5]:
# Filter the dataset to find the cities that fit the criteria.
preferredCitiesDF = cityDataDF.loc[(cityDataDF["Max Temp"]>=minTempInput) & (cityDataDF["Max Temp"]<=maxTempInput)]
if rainDesired:
    preferredCitiesDF = preferredCitiesDF.loc[(cityDataDF["Rain (inches)"] > 0)]
else:
    preferredCitiesDF = preferredCitiesDF.loc[(cityDataDF["Rain (inches)"] == 0.0)]
if snowDesired:
    preferredCitiesDF = preferredCitiesDF.loc[(cityDataDF["Snow (inches)"] > 0)]
else:
    preferredCitiesDF = preferredCitiesDF.loc[(cityDataDF["Snow (inches)"] == 0.0)]   
preferredCitiesDF.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
2,2,Ranaghat,IN,23.18,88.58,82.40,94,75,4.70,haze,0.0,0.0
24,24,Butaritari,KI,3.07,172.79,83.80,76,69,12.03,broken clouds,0.0,0.0
27,27,Sola,VU,-13.88,167.55,80.60,78,90,23.04,overcast clouds,0.0,0.0
29,29,Hilo,US,19.73,-155.09,84.20,58,20,9.17,few clouds,0.0,0.0
42,42,Manuk Mangkaw,PH,4.80,119.85,83.03,71,86,5.26,overcast clouds,0.0,0.0


In [6]:
# Check if enough cities are left after filtering
preferredCitiesDF.count()

City_ID                124
City                   124
Country                124
Lat                    124
Lng                    124
Max Temp               124
Humidity               124
Cloudiness             124
Wind Speed             124
Current Description    124
Rain (inches)          124
Snow (inches)          124
dtype: int64

In [7]:
# Get Hotel Name for the filtered cities
# ------------------------------------------

# Create DataFrame called hotelDF to store hotel names along with city weather info.
hotelDF = preferredCitiesDF[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotelDF["Hotel Name"] = ""
hotelDF.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ranaghat,IN,82.40,haze,23.18,88.58,
24,Butaritari,KI,83.80,broken clouds,3.07,172.79,
27,Sola,VU,80.60,overcast clouds,-13.88,167.55,
29,Hilo,US,84.20,few clouds,19.73,-155.09,
42,Manuk Mangkaw,PH,83.03,overcast clouds,4.80,119.85,


In [8]:
# Set base url
baseUrl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Set parameters to search for a hotel.
params = {"key": gKey, "radius": 5000, "type": "lodging"}

# Iterate through the DataFrame.
for index, row in hotelDF.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(baseUrl, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        # Grab the first hotel from the results and store the name.
        hotelDF.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found in city '{row['City']}'... skipping.")

Hotel not found in city 'Manuk Mangkaw'... skipping.
Hotel not found in city 'Kieta'... skipping.
Hotel not found in city 'Dicabisagan'... skipping.
Hotel not found in city 'Albany'... skipping.
Hotel not found in city 'Gazli'... skipping.
Hotel not found in city 'Tambacounda'... skipping.
Hotel not found in city 'Santa Rosa De Aguan'... skipping.
Hotel not found in city 'Ust-Maya'... skipping.
Hotel not found in city 'Tres Palos'... skipping.
Hotel not found in city 'Ban Nahin'... skipping.
Hotel not found in city 'Marzuq'... skipping.
Hotel not found in city 'Dzhebariki-Khaya'... skipping.
Hotel not found in city 'Chara'... skipping.


In [9]:
hotelDF.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ranaghat,IN,82.40,haze,23.18,88.58,Hotel Pratiksha
24,Butaritari,KI,83.80,broken clouds,3.07,172.79,Isles Sunset Lodge
27,Sola,VU,80.60,overcast clouds,-13.88,167.55,Leumerus Bungalows
29,Hilo,US,84.20,few clouds,19.73,-155.09,Hilo Hawaiian Hotel
42,Manuk Mangkaw,PH,83.03,overcast clouds,4.80,119.85,
43,Hithadhoo,MV,84.56,scattered clouds,-0.60,73.08,Scoop Guest House
57,Atuona,PF,79.52,clear sky,-9.80,-139.03,Villa Enata
62,Gazanjyk,TM,79.12,few clouds,39.24,55.52,Railway motel
63,Riyadh,SA,84.31,scattered clouds,24.69,46.72,InterContinental Riyadh
68,Lorengau,PG,85.59,scattered clouds,-2.02,147.27,Lorengau Harbourside Hotel


In [10]:
# Save DataFrame to a csv file
# --------------------------------------

# Create the output file (CSV).
outputDataFile = os.path.join("data", "WeatherPy_vacation.csv")
# Export the City_Data into a CSV.
hotelDF.to_csv(outputDataFile, index_label="City_ID")

In [11]:
# Locations from hotel DataFrame
# ------------------------------------
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=gKey)
# Center and zoom map appropriately
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

locations = hotelDF[["Lat","Lng"]]

infoBoxTemplate = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

hotelInfo = [infoBoxTemplate.format(**row) for index, row in hotelDF.iterrows()]

markerLayer = gmaps.marker_layer(locations,info_box_content=hotelInfo)
fig.add_layer(markerLayer)

# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))